# Importing the personal statements text file 

This contains 10 BSc Data Science personal statements representating 50% of the cohort starting in 2022

Using NPL the following things will be identified from the personal statements:
- Main topics (Using Topic Modelling)
- Top 10 most frequent words - this may be more insightful than topic modelling as topic modelling is an unsupervised machine learning algorithm it is not always as effective as desired 
- Key people who have been mentioned - this could be used to provide a basis for potential students to do further research to explore their interests
- Readability levels - This will be calculated using the Flesch-Kincaid Grade Level Forumla which calculates the grade level needed to understand a text using the average syllables per word and the average number of words per sentence. 

In [1]:
import pandas as pd

# Reading the text file
with open('ST115_personal_statements.txt', 'r') as file:
    all_ps = file.read()

# Displaying the first 1000 characters of the text file
all_ps[:1000]

'Personal statement: \nTaking part in a coding competition and several UKMT challenges exposed me to new ways of approaching problems from what I was used to in school. The challenge of having to think differently about problems and eventually arrive at solutions after reaching dead-ends was something I relished doing, which convinced me that a quantitative degree was for me.\nI have continued to challenge myself and explore the interplay between computer science and mathematics by solving Project Euler problems. For example, I used recursion to compute the convergents of continued fractions of the square root of 2. These problems nurtured my interest in pure mathematics, leading me to read Kevin Houston\'s "How to Think like a Mathematician". This introduced me to more sophisticated inductive proof techniques than the ones I learned at A-level. To supplement this, I explored how Lean, a programming language, can be taught to inductively prove a simple addition from first principles. W

Looking at the first 1000 characters shown above it can be seen that some intial data cleaning must be done in the sense of removing '\n' and any backslashes in general.
Also we want to manipulate this text data in order to store all the different personal statements as one text to do topic modelling with and to store them as individual statements to calculate readability and key people.

In [2]:
#Inital removal of '\n' and backslashes
all_ps = all_ps.replace("\n", "")
all_ps = all_ps.replace("\xa0", "")
all_ps = all_ps.replace("\'", "")

'Personal statement: Taking part in a coding competition and several UKMT challenges exposed me to new ways of approaching problems from what I was used to in school. The challenge of having to think differently about problems and eventually arrive at solutions after reaching dead-ends was something I relished doing, which convinced me that a quantitative degree was for me.I have continued to challenge myself and explore the interplay between computer science and mathematics by solving Project Euler problems. For example, I used recursion to compute the convergents of continued fractions of the square root of 2. These problems nurtured my interest in pure mathematics, leading me to read Kevin Houstons "How to Think like a Mathematician". This introduced me to more sophisticated inductive proof techniques than the ones I learned at A-level. To supplement this, I explored how Lean, a programming language, can be taught to inductively prove a simple addition from first principles. What en

In [4]:
full_text = all_ps.replace("Personal statement:", " ")

<b> STEP 1: </b>
- Creating a dataframe where each row accounts for a single personal statement. This will be used to calculate the individual readability scores as well as the named entities in each personal statement.

In [5]:

# This phrase has been used in the text file to indicate the start of a different personal statement
individual_statements = all_ps.split('Personal statement:')

df = pd.DataFrame({'Personal Statement': individual_statements})

In [7]:
individual_statements

['',
 ' Taking part in a coding competition and several UKMT challenges exposed me to new ways of approaching problems from what I was used to in school. The challenge of having to think differently about problems and eventually arrive at solutions after reaching dead-ends was something I relished doing, which convinced me that a quantitative degree was for me.I have continued to challenge myself and explore the interplay between computer science and mathematics by solving Project Euler problems. For example, I used recursion to compute the convergents of continued fractions of the square root of 2. These problems nurtured my interest in pure mathematics, leading me to read Kevin Houstons "How to Think like a Mathematician". This introduced me to more sophisticated inductive proof techniques than the ones I learned at A-level. To supplement this, I explored how Lean, a programming language, can be taught to inductively prove a simple addition from first principles. What enthralled me a